# Temporal Convolutional Network used for prediction on time series


In [ ]:
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.8/784.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.0/275.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.optim as optim

from sklearn.metrics import mean_squared_error, mean_absolute_error

from darts import TimeSeries
from darts.models import TCNModel, RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, r2_score, mae, rmse, mse
from darts.utils.missing_values import fill_missing_values
from darts.datasets import AirPassengersDataset, SunspotsDataset, EnergyDataset

In [ ]:
df = pd.read_csv('sensorsRecords_on.csv',  parse_dates=True, encoding = "UTF-8")
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.tz_localize(None)
df['date'] = df['date'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
df = df.groupby(['date'])['reading'].mean().reset_index()

ts = TimeSeries.from_dataframe(df = df, time_col = 'date', value_cols = 'reading', freq = 'H')

hour_series = datetime_attribute_timeseries(ts, attribute='hour', one_hot=True)

train = ts[:-3000]
train_hour = hour_series[:-3000]

val = ts[-3000:]
val_hour = hour_series[-3000:]


ts = fill_missing_values(ts, fill='auto')
train = fill_missing_values(train, fill='auto')
train_hour = fill_missing_values(train_hour, fill='auto')
val = fill_missing_values(val, fill='auto')
val_hour = fill_missing_values(val_hour, fill='auto')


plt.figure(figsize=(12, 6))
plt.plot(ts.pd_dataframe(), label='Reading', color='blue')

plt.title('Sensor Reading')
plt.xlabel('Sample')
plt.ylabel('Sensor Reading')
plt.legend()
plt.show()

In [ ]:
weather_df = pd.read_csv('weather_silo_1.csv',  parse_dates=True)

weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df.set_index('date', inplace=True)

hourly_weather_temp = weather_df['temperature'].resample('H').mean()

plt.figure(figsize=(12, 6))
plt.plot(hourly_weather_temp, label='Weather temperature', color='salmon')
plt.plot(df['reading'], label='Sensor temperature', color='blue')

plt.title('Sensor Reading vs Weather Data')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

# Model 1

In [1]:
# model_0_1 = TCNModel(
#     input_chunk_length=25,
#     output_chunk_length=12
#     )

model_1 = TCNModel(
    input_chunk_length=25,
    output_chunk_length=12,
    n_epochs=1000,
    dropout=0.1,
    dilation_base=2,
    weight_norm=True,
    kernel_size=5,
    num_filters=64,
    random_state=0,
    model_name='model1_1000epochs',
    save_checkpoints=True
)

In [ ]:
model_1.fit(series=train,
              past_covariates=train_hour,
              val_series=val,
              val_past_covariates=val_hour,
              verbose=True)

In [ ]:
backtest = model_1.historical_forecasts(series=val,
                                          past_covariates=hour_series,
                                          forecast_horizon=6,
                                          retrain=False,
                                          verbose=False)

In [ ]:
backtest = fill_missing_values(backtest, fill='auto')
predictions = backtest.pd_dataframe()
predictions = predictions['reading'].values

val1 = val.pd_dataframe()
val1 = val1['reading'].values

plt.figure(figsize=(12, 6))
time_steps = range(len(predictions))

plt.plot(range(len(val1)), val1, label='Actual Data', color='blue')
plt.plot(range(len(predictions)), predictions, label='Predictions', color='red')

plt.title('Sensor Reading Prediction vs Actual Data')
plt.xlabel('Sample')
plt.ylabel('Sensor Reading')
plt.legend()
y_values = [i for i in range(0, 50, 2)]
plt.yticks(y_values)
plt.show()

In [ ]:
mse = mean_squared_error(predictions, val1)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE): ", rmse)

mean_squared = mean_squared_error(val1, predictions)
print('Mean Squared Error (MSE):', mean_squared)

mean_absolute = mean_absolute_error(val1, predictions)
print('Mean Absolute Error (MAE):', mean_absolute)

from statsmodels.tsa.stattools import adfuller

#ADF test on 'y'
result_y = adfuller(val1)
print("ADF Statistic for 'y':", result_y[0])
print("p-value for 'y':", result_y[1])
print("Critical Values for 'y':", result_y[4])

# Model 2

In [ ]:
model_2 = TCNModel(
    input_chunk_length=25,
    output_chunk_length=12,
    n_epochs=1000,
    dropout=0.4,
    dilation_base=16,
    weight_norm=True,
    kernel_size=7,
    num_filters=128,
    random_state=0,
    model_name='model2_1000epochs',
    save_checkpoints=True
)

In [ ]:
model_2.fit(series=train,
              past_covariates=train_hour,
              val_series=val,
              val_past_covariates=val_hour,
              verbose=True)

In [ ]:
backtest = model_2.historical_forecasts(series=val,
                                          past_covariates=hour_series,
                                          forecast_horizon = 6,
                                          retrain=False,
                                          verbose=False)

In [ ]:
backtest = fill_missing_values(backtest, fill='auto')
predictions = backtest.pd_dataframe()
predictions = predictions['reading'].values

val1 = val.pd_dataframe()
val1 = val1['reading'].values

plt.figure(figsize=(12, 6))

plt.plot(range(len(val1)), val1, label='Actual Data', color='blue')
plt.plot(range(len(predictions)), predictions, label='Predictions', color='red')

plt.title('Sensor Reading Prediction vs Actual Data')
plt.xlabel('Sample')
plt.ylabel('Sensor Reading')
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(predictions, val1)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE): ", rmse)

mean_squared = mean_squared_error(val1, predictions)
print('Mean Squared Error (MSE):', mean_squared)

mean_absolute = mean_absolute_error(val1, predictions)
print('Mean Absolute Error (MAE):', mean_absolute)

from statsmodels.tsa.stattools import adfuller

#ADF test on 'y'
result_y = adfuller(val1)
print("ADF Statistic for 'y':", result_y[0])
print("p-value for 'y':", result_y[1])
print("Critical Values for 'y':", result_y[4])

# Model 3

In [ ]:
model_3 = TCNModel(
    input_chunk_length=25,
    output_chunk_length=12,
    n_epochs=1000,
    dropout=0.8,
    dilation_base=8,
    weight_norm=True,
    kernel_size=3,
    num_filters=32,
    random_state=0,
    model_name='model3_1000epochs',
    save_checkpoints=True
)

In [ ]:
model_3.fit(series=train,
              past_covariates=train_hour,
              val_series=val,
              val_past_covariates=val_hour,
              verbose=True)

In [ ]:
backtest = model_3.historical_forecasts(series=val,
                                          past_covariates=hour_series,
                                          forecast_horizon = 6,
                                          retrain=False,
                                          verbose=False)

In [ ]:
backtest = fill_missing_values(backtest, fill='auto')
predictions = backtest.pd_dataframe()
predictions = predictions['reading'].values

val1 = val.pd_dataframe()
val1 = val1['reading'].values

plt.figure(figsize=(12, 6))

plt.plot(range(len(val1)), val1, label='Actual Data', color='blue')
plt.plot(range(len(predictions)), predictions, label='Predictions', color='red')

plt.title('Sensor Reading Prediction vs Actual Data')
plt.xlabel('Sample')
plt.ylabel('Sensor Reading')
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(predictions, val1)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE): ", rmse)

mean_squared = mean_squared_error(val1, predictions)
print('Mean Squared Error (MSE):', mean_squared)

mean_absolute = mean_absolute_error(val1, predictions)
print('Mean Absolute Error (MAE):', mean_absolute)

from statsmodels.tsa.stattools import adfuller

#ADF test on 'y'
result_y = adfuller(val1)
print("ADF Statistic for 'y':", result_y[0])
print("p-value for 'y':", result_y[1])
print("Critical Values for 'y':", result_y[4])